参考:<https://www.cnblogs.com/leohahah/p/10189281.html>

## 一.基础概念
在进行具体的迭代类和迭代函数示例之前，首先介绍几个概念以防混淆：

### 可迭代对象iterable
可迭代的对象的意思是就是说这个实体是可迭代的，例如字符、列表、元组、字典、迭代器等等，可以用for ... in进行循环，可以使用for循环迭代的标志是内部实现了`__iter__`方法。

可迭代对象仅含有`__iter__`的内部方法，你可以通过封装`next()`方法（python3中为`__next__()`）来将其做成一个迭代器，以生成器（generator，特殊的函数类型的迭代器）为例，你可以通过yield关键字来做一个迭代器，只不过名字被叫做generator，yield可以看做就是为对象添加了`__iter__`方法和指示下一次迭代的`next()`/`__next__()`方法。

使用`isinstance(实体名,Iterable)`可判断是否为可迭代对象

### 迭代器iterator

迭代器就是实现了迭代方式的容器，iterable对象一般只能按默认的正序方式进行迭代，你可以通过为其添加`__next__()`/next()方法来定制不同的迭代方式，这样通过next方法封装起来的迭代对象生成器就被称作迭代器。与iterable相比,iterator多了一个next()方法，next()方法用于定制for循环时具体的返回值及返回顺序以及处理StopIteration异常等。

iterator必定是iterable的，因此其必然含有`__iter__`方法，此方法保证了iterator是可以迭代的，个人认为可以将`__iter__`()方法看做迭代器的入口，此入口告诉python对象是可for循环的，当你还为class定义了`__next__`方法时,python的for循环会直接调用`__next__()`方法进行迭代，因此对于实现了`__next__`方法的迭代器来讲`__iter__`方法是一个不可或缺的鸡肋方法，不可或缺是因为他是可迭代的标识，鸡肋是因为他不会实质性的起作用，虽然他是迭代器的入口但却不是迭代的起始点，也因此iterator的`__iter__`方法可以随意返回self或者self.属性或者None。

使用`isinstance(实体名,Iterator)`可判断是否为迭代器

### 生成器generator

generator对象是一种特殊的iterator函数，它会在执行过程中保存执行的上下文环境，并在下次循环中从yield语句后继续执行，生成器的标志就是yield关键字。

generator不需要抛出StopIteration异常（你可以看做yield已经在内部实现了StopIteration跳出循环），函数并没有将序列项一次生成，所以generator在实现上可以有无穷个元素，而不需要无穷的存储空间，这在内存优化方面很有用处。

使用isinstance(实体名,Generator)可判断是否为生成器。

生成器的创建办法有两种：
- 通过函数创建，称作generator function
- 通过推导式创建，例如`g=(x*2 for x in range(10))`，称作generator expression


### `__iter__()`和iter()

python有一个built-in函数iter()用来从序列对象，如String, list,tuple中生成迭代器。

`__iter__`()方法是python的魔法方法，如果对象是iterator，那么for循环时，python会直接调用`__next__`()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

在python中，如果对象不含`__next__`方法，但是`__iter__`只返回self的话，尝试对对象使用for循环就会报“TypeError: iter() returned non-iterator of type [类名]”，针对这种错误要么加一个`__next__`()方法，要么`__iter__`()返回一个包含`__next__`()方法的迭代器对象。

那么按理来说string、list这些iterable对象也是只含有`__iter__`不含`__next__`的，为何他们就可以for循环呢，这点在本文中的示例三中有演示，如果`__iter__`魔法方法调用了iter()方法，返回了一个迭代器对象，那么即便其不包含`__next__`也可以进行迭代

## 二.实例

In [13]:
from collections import Iterable,Iterator,Generator

lst = ['one','two' , 'three', 'four']
print('Iterable 判断:', isinstance(lst, Iterable))
print('Iterator 判断:', isinstance(lst, Iterator))
print('Generator 判断:', isinstance(lst, Generator))

Iterable 判断: True
Iterator 判断: False
Generator 判断: False


可以看到这是一个可迭代对象但并不是迭代器，我们把它搞成一个迭代器试试看：

### 注意:generator对象是一种特殊的iterator函数

In [35]:
# 方法一,使用yield关键字：

def lst_to_generator(lst):
    for num in lst:
        yield num

lst_changed = lst_to_generator(lst) 

for num in lst_changed:
    print('Number is:', num)

print('lst改造后,Iterable 判断:', isinstance(lst_changed, Iterable))
print('lst改造后,Iterator 判断:', isinstance(lst_changed, Iterator))
print('lst改造后,Generator 判断:', isinstance(lst_changed, Generator))

# 改造成了生成器

Number is: one
Number is: two
Number is: three
Number is: four
lst改造后,Iterable 判断: True
lst改造后,Iterator 判断: True
lst改造后,Generator 判断: True


In [36]:
# 方法二,增加__iter__(),__next__():

class lst_to_iterator(object):
    def __init__(self, lst):
        self.lst = lst
        self.len = len(self.lst)
        self.cur_index = -1
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self.cur_index += 1
        if self.cur_index < self.len:
            return self.lst[self.cur_index]
        else:
            raise StopIteration

lst_changed = lst_to_iterator(lst) 

for num in lst_to_iterator(lst):
    print('Number is:', num)

print('lst改造后,Iterable 判断:', isinstance(lst_changed, Iterable))
print('lst改造后,Iterator 判断:', isinstance(lst_changed, Iterator))
print('lst改造后,Generator 判断:', isinstance(lst_changed, Generator))  # 不是生成器

Number is: one
Number is: two
Number is: three
Number is: four
lst改造后,Iterable 判断: True
lst改造后,Iterator 判断: True
lst改造后,Generator 判断: False


### 问题一：既然可迭代对象也可以使用for循环遍历，为何还要使用迭代器呢？

一般情况下不需要将可迭代对象封装为迭代器。但是想象一种需要重复迭代的场景，在一个class中我们需要对输入数组进行正序、反序、正序step=1、正序step=2等等等等的多种重复遍历，那么我们完全可以针对每一种遍历方式写一个迭代容器，这样就不用每次需要遍历时都费劲心思的写一堆对应的for循环代码，只要调用相应名称的迭代器就能做到，针对每一种迭代器我们还可以加上类型判断及相应的处理，这使得我们可以不必关注底层的迭代代码实现。

从这种角度来看，你可以将迭代器看做可迭代对象的函数化，有一个非常流行的迭代器库itertools，其实就是如上所说的，他为很多可迭代类型提前定义好了一些列的常见迭代方式，并封装为了迭代器，这样大家就可以很方便的直接通过调用此模块玩转迭代。

此外iterator还可以节省内存

In [46]:
# 练习1:
lst2 = [1, 2, 3, 4, 6, 5, 8, 9, 7]

class iter_sort(object):
    """
        index正序迭代器
    @object: 可迭代对象
    @return: 返回正序后的结果,元素与原lst结果一致
    """
    def __init__(self, lst):
        self.lst = lst
        self.len = len(lst)
        self.cur_index = -1
        
    def __iter__(self):
        return self
    
    def __next__(self):
        self.cur_index += 1
        if self.cur_index < self.len:
            return self.lst[self.cur_index]
        else:
            raise StopIteration


class iter_reverse(object):
    """
        index倒序迭代器
    @object: 可迭代对象
    @return: 返回倒序后的结果,元素与原lst顺序相反
    """
    def __init__(self, lst):
        self.lst = lst
        self.len = len(self.lst)
        self.cur_index = self.len
                
    def __iter__(self):
        return self
    
    def __next__(self):
        self.cur_index -= 1
        if self.cur_index >= 0:
            return self.lst[self.cur_index]
        else:
            raise StopIteration

lst_sort = iter_sort(lst2)
lst_reverse = iter_reverse(lst2)

print("lst_sort", list(lst_sort))
print("lst_reverse", list(lst_reverse))

lst_sort [1, 2, 3, 4, 6, 5, 8, 9, 7]
lst_reverse [7, 9, 8, 5, 6, 4, 3, 2, 1]


可以看到我们只要调用相应名字的迭代器对象就可以直接进行for循环了，这种写法相比起每次都需要在for循环中重复一遍算法逻辑要简单，除此之外你还可以为不同输入类型定制相同的迭代方式，这样就无需关注内部实现了。

这就是迭代器的作用，为不同类型的输入封装相同的迭代功能，从而实现代码简化。

Python中有一个非常有用的itertools module，提供了大量不同的迭代器，只要直接调用你就可以实现对序列的各种操作，你可以通过这个库加深对于迭代器的理解

### 问题二：生成器（generator）如何节约内存的？

generator的标志性关键字yield其实可以看作return，以本文上述的lst_to_generator方法为例，lst_to_generator(lst)就是一个生成器。

生成器最大的好处在于：lst_to_generator(lst)并不会真正执行函数的代码，只有在被循环时才会去获取值，且每次循环都return一个值（即yield一个值），在处理完毕后下次循环时依然使用相同的内存（假设处理单位大小一样）来获取值并处理，这样在一次for循环中函数好像中断（yield）了无数次，每次都用相同大小的内存来存储被迭代的值。

yield与return的最大区别就是yield并不意味着函数的终止，而是意味着函数的一次中断，在未被迭代完毕之前yield意味着先返回一次迭代值并继续下一次函数的执行（起始位置是上一次yeild语句结束），而return则基本意味着一个函数的彻底终止并返回一个全量的返回值。

因此generator是为了节省内存的，而且将函数写为一个生成器可以使函数变的可迭代，如果我们想遍历函数的返回值，我们不用再单独定义一个可迭代变量存储函数的返回值们，而是直接迭代生成器函数即可（除非函数本身返回一个全量的可迭代对象）。

同理iterator的`__iter__`()方法只是一个迭代的入口，每次调用`__next__()`时返回一个迭代值，同样以O(1)的空间复杂度完成了迭代。

In [77]:
import time
# 练习2:

def test():
    begin_time = time.time()
    result = []
    for i in range(10000000):  
        if i % 2 ==0:
            result.append(i)  # for循环结束后,会生成一个含有10000000个数的list了，占内存
    end_time = time.time()
    print(end_time - begin_time)
    return result


# 用yield实现
def test2():
    begin_time = time.time()
    for i in range(10000000):
        if i % 2 == 0:
            yield i   # 一次只生成1个数的内存
    end_time = time.time()
    print(end_time - begin_time)

test()
list(test2())[:10]

# yield不仅节省内存,同时也比较省时

1.003375768661499
0.9746119976043701


[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

可迭代对象(但并不是迭代器),其标志不仅仅是含有`__iter__`方法，它的`__iter__`方法还返回了一个迭代器对象（例如练习3中的iter(self.list)），

所以虽然其可for循环,但本身却不含`__next__()`,因此并不是迭代器

In [60]:
# 练习3:

class iter_fake(object):
    """
        没有__next__方法,但确实能够迭代数据
    """
    def __init__(self, lst):
        self.lst = lst
        
    def __iter__(self):
        return iter(self.lst)

result = iter_fake(lst2)

print('iter_fake可迭代:', list(result))

print('iter_fake,Iterable 判断:', isinstance(result, Iterable))
print('iter_fake,Iterator 判断:', isinstance(result, Iterator))  # isinstance并不认为iter_fake类是一个迭代器.但其仍是可迭代的


iter_fake可迭代: [1, 2, 3, 4, 6, 5, 8, 9, 7]
iter_fake,Iterable 判断: True
iter_fake,Iterator 判断: False
